In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

Using TensorFlow backend.


In [2]:
from tensorflow.python.keras.backend import set_session
from tensorflow.python.keras.models import load_model

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

sess = tf.Session()
graph = tf.get_default_graph()

# IMPORTANT: models have to be loaded AFTER SETTING THE SESSION for keras! 
# Otherwise, their weights will be unavailable in the threads after the session there has been set
set_session(sess)

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch>=19000:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig(r"C:\Users\rishi\Documents\DCGAN Images\mnist_%d.png" % epoch)
        plt.close()


In [5]:
global sess
global graph
with graph.as_default():
    set_session(sess)
    dcgan = DCGAN()
    dcgan.train(epochs=20000, batch_size=32, save_interval=200)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 8, 8, 64)         

c:\users\rishi\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
c:\users\rishi\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 1.254443, acc.: 21.88%] [G loss: 0.642583]


c:\users\rishi\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 0.730405, acc.: 62.50%] [G loss: 1.206398]
2 [D loss: 0.982817, acc.: 48.44%] [G loss: 1.233961]
3 [D loss: 0.786095, acc.: 54.69%] [G loss: 1.200321]
4 [D loss: 0.601307, acc.: 71.88%] [G loss: 1.541217]
5 [D loss: 0.615374, acc.: 67.19%] [G loss: 0.911633]
6 [D loss: 0.428953, acc.: 85.94%] [G loss: 1.044277]
7 [D loss: 0.431989, acc.: 78.12%] [G loss: 0.793195]
8 [D loss: 0.479585, acc.: 79.69%] [G loss: 0.724335]
9 [D loss: 0.549037, acc.: 78.12%] [G loss: 0.490069]
10 [D loss: 0.651313, acc.: 70.31%] [G loss: 0.852947]
11 [D loss: 0.730205, acc.: 65.62%] [G loss: 0.856025]
12 [D loss: 0.684466, acc.: 60.94%] [G loss: 1.157683]
13 [D loss: 0.919807, acc.: 40.62%] [G loss: 1.289965]
14 [D loss: 0.688921, acc.: 56.25%] [G loss: 1.974037]
15 [D loss: 0.682638, acc.: 62.50%] [G loss: 1.769087]
16 [D loss: 0.738739, acc.: 64.06%] [G loss: 1.868061]
17 [D loss: 0.599193, acc.: 62.50%] [G loss: 2.457524]
18 [D loss: 0.617482, acc.: 65.62%] [G loss: 2.201968]
19 [D loss: 0.51450

...

19331 [D loss: 0.647604, acc.: 65.62%] [G loss: 1.134113]
19332 [D loss: 0.709501, acc.: 50.00%] [G loss: 0.930026]
19333 [D loss: 0.311766, acc.: 95.31%] [G loss: 1.172865]
19334 [D loss: 0.735762, acc.: 50.00%] [G loss: 0.981734]
19335 [D loss: 0.569924, acc.: 75.00%] [G loss: 1.092742]
19336 [D loss: 0.761895, acc.: 51.56%] [G loss: 1.249433]
19337 [D loss: 0.936791, acc.: 29.69%] [G loss: 1.093238]
19338 [D loss: 0.844488, acc.: 46.88%] [G loss: 1.083233]
19339 [D loss: 0.680210, acc.: 57.81%] [G loss: 1.343099]
19340 [D loss: 0.751307, acc.: 48.44%] [G loss: 0.851092]
19341 [D loss: 0.518915, acc.: 78.12%] [G loss: 1.073547]
19342 [D loss: 0.542629, acc.: 73.44%] [G loss: 1.008229]
19343 [D loss: 0.627609, acc.: 67.19%] [G loss: 1.249364]
19344 [D loss: 0.730131, acc.: 59.38%] [G loss: 0.999567]
19345 [D loss: 0.593985, acc.: 70.31%] [G loss: 1.041688]
19346 [D loss: 0.967372, acc.: 37.50%] [G loss: 0.618933]
19347 [D loss: 0.636900, acc.: 65.62%] [G loss: 1.582051]
19348 [D loss:

19473 [D loss: 0.682850, acc.: 56.25%] [G loss: 0.947235]
19474 [D loss: 0.448602, acc.: 84.38%] [G loss: 1.272059]
19475 [D loss: 0.721726, acc.: 53.12%] [G loss: 0.725398]
19476 [D loss: 0.739283, acc.: 56.25%] [G loss: 1.010540]
19477 [D loss: 0.669151, acc.: 54.69%] [G loss: 1.208947]
19478 [D loss: 0.556409, acc.: 73.44%] [G loss: 1.372602]
19479 [D loss: 0.529025, acc.: 79.69%] [G loss: 1.227424]
19480 [D loss: 0.688831, acc.: 65.62%] [G loss: 1.311588]
19481 [D loss: 0.610889, acc.: 67.19%] [G loss: 1.460109]
19482 [D loss: 0.536577, acc.: 70.31%] [G loss: 0.986199]
19483 [D loss: 0.497862, acc.: 79.69%] [G loss: 1.489854]
19484 [D loss: 0.521541, acc.: 76.56%] [G loss: 1.244556]
19485 [D loss: 0.711591, acc.: 51.56%] [G loss: 0.723456]
19486 [D loss: 0.551232, acc.: 73.44%] [G loss: 1.287643]
19487 [D loss: 0.634763, acc.: 62.50%] [G loss: 1.548052]
19488 [D loss: 0.679220, acc.: 59.38%] [G loss: 1.339826]
19489 [D loss: 0.654761, acc.: 65.62%] [G loss: 0.847979]
19490 [D loss:

19615 [D loss: 0.782099, acc.: 53.12%] [G loss: 0.768499]
19616 [D loss: 0.582756, acc.: 71.88%] [G loss: 1.250299]
19617 [D loss: 0.631412, acc.: 60.94%] [G loss: 1.630171]
19618 [D loss: 0.558657, acc.: 70.31%] [G loss: 0.957337]
19619 [D loss: 0.691959, acc.: 54.69%] [G loss: 1.356620]
19620 [D loss: 0.578997, acc.: 64.06%] [G loss: 1.326799]
19621 [D loss: 0.638392, acc.: 64.06%] [G loss: 1.091933]
19622 [D loss: 0.453072, acc.: 82.81%] [G loss: 1.121039]
19623 [D loss: 0.880958, acc.: 34.38%] [G loss: 1.312668]
19624 [D loss: 0.624799, acc.: 60.94%] [G loss: 0.956678]
19625 [D loss: 0.685768, acc.: 59.38%] [G loss: 0.834633]
19626 [D loss: 0.496426, acc.: 79.69%] [G loss: 1.180578]
19627 [D loss: 0.578753, acc.: 70.31%] [G loss: 1.269753]
19628 [D loss: 0.649075, acc.: 60.94%] [G loss: 1.190835]
19629 [D loss: 0.687958, acc.: 57.81%] [G loss: 0.987433]
19630 [D loss: 0.588303, acc.: 67.19%] [G loss: 1.051615]
19631 [D loss: 0.701875, acc.: 62.50%] [G loss: 1.697317]
19632 [D loss:

19757 [D loss: 0.552137, acc.: 73.44%] [G loss: 1.547267]
19758 [D loss: 0.546807, acc.: 68.75%] [G loss: 1.533050]
19759 [D loss: 0.710749, acc.: 54.69%] [G loss: 1.456762]
19760 [D loss: 0.471431, acc.: 81.25%] [G loss: 1.190251]
19761 [D loss: 0.599472, acc.: 70.31%] [G loss: 0.909268]
19762 [D loss: 0.687623, acc.: 59.38%] [G loss: 1.148290]
19763 [D loss: 0.582272, acc.: 68.75%] [G loss: 1.718252]
19764 [D loss: 0.708738, acc.: 53.12%] [G loss: 1.146044]
19765 [D loss: 0.686809, acc.: 65.62%] [G loss: 1.239030]
19766 [D loss: 0.779867, acc.: 53.12%] [G loss: 1.101506]
19767 [D loss: 0.510987, acc.: 73.44%] [G loss: 1.048375]
19768 [D loss: 0.611509, acc.: 70.31%] [G loss: 0.734887]
19769 [D loss: 0.585076, acc.: 67.19%] [G loss: 1.091399]
19770 [D loss: 0.631543, acc.: 68.75%] [G loss: 1.075815]
19771 [D loss: 0.518459, acc.: 76.56%] [G loss: 1.357091]
19772 [D loss: 0.682262, acc.: 59.38%] [G loss: 1.138710]
19773 [D loss: 0.555873, acc.: 70.31%] [G loss: 0.837523]
19774 [D loss:

19899 [D loss: 0.581421, acc.: 71.88%] [G loss: 1.331534]
19900 [D loss: 0.526897, acc.: 73.44%] [G loss: 0.929937]
19901 [D loss: 0.778116, acc.: 48.44%] [G loss: 1.341197]
19902 [D loss: 0.723131, acc.: 53.12%] [G loss: 1.558113]
19903 [D loss: 0.602360, acc.: 71.88%] [G loss: 1.190372]
19904 [D loss: 0.735758, acc.: 57.81%] [G loss: 0.928445]
19905 [D loss: 0.595723, acc.: 68.75%] [G loss: 1.099088]
19906 [D loss: 0.652236, acc.: 57.81%] [G loss: 0.964547]
19907 [D loss: 0.461803, acc.: 84.38%] [G loss: 1.280070]
19908 [D loss: 0.558809, acc.: 78.12%] [G loss: 0.840091]
19909 [D loss: 0.690545, acc.: 59.38%] [G loss: 1.068711]
19910 [D loss: 0.811151, acc.: 48.44%] [G loss: 1.302919]
19911 [D loss: 0.569337, acc.: 68.75%] [G loss: 1.279868]
19912 [D loss: 0.781252, acc.: 46.88%] [G loss: 1.005059]
19913 [D loss: 0.635603, acc.: 59.38%] [G loss: 1.266363]
19914 [D loss: 0.722565, acc.: 43.75%] [G loss: 1.385941]
19915 [D loss: 0.909584, acc.: 31.25%] [G loss: 1.053418]
19916 [D loss:

In [6]:
def generate_images(cnt):
    c=1
    while(c <= cnt):
        noise = np.random.normal(0, 1, (1, 100))
        gen_imgs = dcgan.generator.predict(noise)
        gen_imgs = 0.5 * gen_imgs + 0.5
        plt.imshow(gen_imgs[0, :,:,0], cmap='gray')
        plt.axis('off')
        plt.savefig(r"C:\Users\rishi\Documents\DCGAN Images\generated_dcgan\dcgan_{}.jpg".format(c), bbox_inches='tight', pad_inches=-0.0005)
        print("Image {} saved.".format(c))
        c+=1
        plt.close()
generate_images(10000)

Image 1 saved.
Image 2 saved.
Image 3 saved.
Image 4 saved.
Image 5 saved.
Image 6 saved.
Image 7 saved.
Image 8 saved.
Image 9 saved.
Image 10 saved.
Image 11 saved.
Image 12 saved.
Image 13 saved.
Image 14 saved.
Image 15 saved.
Image 16 saved.
Image 17 saved.
Image 18 saved.
Image 19 saved.
Image 20 saved.
Image 21 saved.
Image 22 saved.
Image 23 saved.
Image 24 saved.
Image 25 saved.
Image 26 saved.
Image 27 saved.
Image 28 saved.
Image 29 saved.
Image 30 saved.
Image 31 saved.
Image 32 saved.
Image 33 saved.
Image 34 saved.
Image 35 saved.
Image 36 saved.
Image 37 saved.
Image 38 saved.
Image 39 saved.
Image 40 saved.
Image 41 saved.
Image 42 saved.
Image 43 saved.
Image 44 saved.
Image 45 saved.
Image 46 saved.
Image 47 saved.
Image 48 saved.
Image 49 saved.
Image 50 saved.
Image 51 saved.
Image 52 saved.
Image 53 saved.
Image 54 saved.
Image 55 saved.
Image 56 saved.
Image 57 saved.
Image 58 saved.
Image 59 saved.
Image 60 saved.
Image 61 saved.
Image 62 saved.
Image 63 saved.
I

Image 491 saved.
Image 492 saved.
Image 493 saved.
Image 494 saved.
Image 495 saved.
Image 496 saved.
Image 497 saved.
Image 498 saved.
Image 499 saved.
Image 500 saved.
Image 501 saved.
Image 502 saved.
Image 503 saved.
Image 504 saved.
Image 505 saved.
Image 506 saved.
Image 507 saved.
Image 508 saved.
Image 509 saved.
Image 510 saved.
Image 511 saved.
Image 512 saved.
Image 513 saved.
Image 514 saved.
Image 515 saved.
Image 516 saved.
Image 517 saved.
Image 518 saved.
Image 519 saved.
Image 520 saved.
Image 521 saved.
Image 522 saved.
Image 523 saved.
Image 524 saved.
Image 525 saved.
Image 526 saved.
Image 527 saved.
Image 528 saved.
Image 529 saved.
Image 530 saved.
Image 531 saved.
Image 532 saved.
Image 533 saved.
Image 534 saved.
Image 535 saved.
Image 536 saved.
Image 537 saved.
Image 538 saved.
Image 539 saved.
Image 540 saved.
Image 541 saved.
Image 542 saved.
Image 543 saved.
Image 544 saved.
Image 545 saved.
Image 546 saved.
Image 547 saved.
Image 548 saved.
Image 549 save

Image 973 saved.
Image 974 saved.
Image 975 saved.
Image 976 saved.
Image 977 saved.
Image 978 saved.
Image 979 saved.
Image 980 saved.
Image 981 saved.
Image 982 saved.
Image 983 saved.
Image 984 saved.
Image 985 saved.
Image 986 saved.
Image 987 saved.
Image 988 saved.
Image 989 saved.
Image 990 saved.
Image 991 saved.
Image 992 saved.
Image 993 saved.
Image 994 saved.
Image 995 saved.
Image 996 saved.
Image 997 saved.
Image 998 saved.
Image 999 saved.
Image 1000 saved.
Image 1001 saved.
Image 1002 saved.
Image 1003 saved.
Image 1004 saved.
Image 1005 saved.
Image 1006 saved.
Image 1007 saved.
Image 1008 saved.
Image 1009 saved.
Image 1010 saved.
Image 1011 saved.
Image 1012 saved.
Image 1013 saved.
Image 1014 saved.
Image 1015 saved.
Image 1016 saved.
Image 1017 saved.
Image 1018 saved.
Image 1019 saved.
Image 1020 saved.
Image 1021 saved.
Image 1022 saved.
Image 1023 saved.
Image 1024 saved.
Image 1025 saved.
Image 1026 saved.
Image 1027 saved.
Image 1028 saved.
Image 1029 saved.
I

Image 1430 saved.
Image 1431 saved.
Image 1432 saved.
Image 1433 saved.
Image 1434 saved.
Image 1435 saved.
Image 1436 saved.
Image 1437 saved.
Image 1438 saved.
Image 1439 saved.
Image 1440 saved.
Image 1441 saved.
Image 1442 saved.
Image 1443 saved.
Image 1444 saved.
Image 1445 saved.
Image 1446 saved.
Image 1447 saved.
Image 1448 saved.
Image 1449 saved.
Image 1450 saved.
Image 1451 saved.
Image 1452 saved.
Image 1453 saved.
Image 1454 saved.
Image 1455 saved.
Image 1456 saved.
Image 1457 saved.
Image 1458 saved.
Image 1459 saved.
Image 1460 saved.
Image 1461 saved.
Image 1462 saved.
Image 1463 saved.
Image 1464 saved.
Image 1465 saved.
Image 1466 saved.
Image 1467 saved.
Image 1468 saved.
Image 1469 saved.
Image 1470 saved.
Image 1471 saved.
Image 1472 saved.
Image 1473 saved.
Image 1474 saved.
Image 1475 saved.
Image 1476 saved.
Image 1477 saved.
Image 1478 saved.
Image 1479 saved.
Image 1480 saved.
Image 1481 saved.
Image 1482 saved.
Image 1483 saved.
Image 1484 saved.
Image 1485

Image 1885 saved.
Image 1886 saved.
Image 1887 saved.
Image 1888 saved.
Image 1889 saved.
Image 1890 saved.
Image 1891 saved.
Image 1892 saved.
Image 1893 saved.
Image 1894 saved.
Image 1895 saved.
Image 1896 saved.
Image 1897 saved.
Image 1898 saved.
Image 1899 saved.
Image 1900 saved.
Image 1901 saved.
Image 1902 saved.
Image 1903 saved.
Image 1904 saved.
Image 1905 saved.
Image 1906 saved.
Image 1907 saved.
Image 1908 saved.
Image 1909 saved.
Image 1910 saved.
Image 1911 saved.
Image 1912 saved.
Image 1913 saved.
Image 1914 saved.
Image 1915 saved.
Image 1916 saved.
Image 1917 saved.
Image 1918 saved.
Image 1919 saved.
Image 1920 saved.
Image 1921 saved.
Image 1922 saved.
Image 1923 saved.
Image 1924 saved.
Image 1925 saved.
Image 1926 saved.
Image 1927 saved.
Image 1928 saved.
Image 1929 saved.
Image 1930 saved.
Image 1931 saved.
Image 1932 saved.
Image 1933 saved.
Image 1934 saved.
Image 1935 saved.
Image 1936 saved.
Image 1937 saved.
Image 1938 saved.
Image 1939 saved.
Image 1940

...

Image 7832 saved.
Image 7833 saved.
Image 7834 saved.
Image 7835 saved.
Image 7836 saved.
Image 7837 saved.
Image 7838 saved.
Image 7839 saved.
Image 7840 saved.
Image 7841 saved.
Image 7842 saved.
Image 7843 saved.
Image 7844 saved.
Image 7845 saved.
Image 7846 saved.
Image 7847 saved.
Image 7848 saved.
Image 7849 saved.
Image 7850 saved.
Image 7851 saved.
Image 7852 saved.
Image 7853 saved.
Image 7854 saved.
Image 7855 saved.
Image 7856 saved.
Image 7857 saved.
Image 7858 saved.
Image 7859 saved.
Image 7860 saved.
Image 7861 saved.
Image 7862 saved.
Image 7863 saved.
Image 7864 saved.
Image 7865 saved.
Image 7866 saved.
Image 7867 saved.
Image 7868 saved.
Image 7869 saved.
Image 7870 saved.
Image 7871 saved.
Image 7872 saved.
Image 7873 saved.
Image 7874 saved.
Image 7875 saved.
Image 7876 saved.
Image 7877 saved.
Image 7878 saved.
Image 7879 saved.
Image 7880 saved.
Image 7881 saved.
Image 7882 saved.
Image 7883 saved.
Image 7884 saved.
Image 7885 saved.
Image 7886 saved.
Image 7887

Image 8289 saved.
Image 8290 saved.
Image 8291 saved.
Image 8292 saved.
Image 8293 saved.
Image 8294 saved.
Image 8295 saved.
Image 8296 saved.
Image 8297 saved.
Image 8298 saved.
Image 8299 saved.
Image 8300 saved.
Image 8301 saved.
Image 8302 saved.
Image 8303 saved.
Image 8304 saved.
Image 8305 saved.
Image 8306 saved.
Image 8307 saved.
Image 8308 saved.
Image 8309 saved.
Image 8310 saved.
Image 8311 saved.
Image 8312 saved.
Image 8313 saved.
Image 8314 saved.
Image 8315 saved.
Image 8316 saved.
Image 8317 saved.
Image 8318 saved.
Image 8319 saved.
Image 8320 saved.
Image 8321 saved.
Image 8322 saved.
Image 8323 saved.
Image 8324 saved.
Image 8325 saved.
Image 8326 saved.
Image 8327 saved.
Image 8328 saved.
Image 8329 saved.
Image 8330 saved.
Image 8331 saved.
Image 8332 saved.
Image 8333 saved.
Image 8334 saved.
Image 8335 saved.
Image 8336 saved.
Image 8337 saved.
Image 8338 saved.
Image 8339 saved.
Image 8340 saved.
Image 8341 saved.
Image 8342 saved.
Image 8343 saved.
Image 8344

Image 8747 saved.
Image 8748 saved.
Image 8749 saved.
Image 8750 saved.
Image 8751 saved.
Image 8752 saved.
Image 8753 saved.
Image 8754 saved.
Image 8755 saved.
Image 8756 saved.
Image 8757 saved.
Image 8758 saved.
Image 8759 saved.
Image 8760 saved.
Image 8761 saved.
Image 8762 saved.
Image 8763 saved.
Image 8764 saved.
Image 8765 saved.
Image 8766 saved.
Image 8767 saved.
Image 8768 saved.
Image 8769 saved.
Image 8770 saved.
Image 8771 saved.
Image 8772 saved.
Image 8773 saved.
Image 8774 saved.
Image 8775 saved.
Image 8776 saved.
Image 8777 saved.
Image 8778 saved.
Image 8779 saved.
Image 8780 saved.
Image 8781 saved.
Image 8782 saved.
Image 8783 saved.
Image 8784 saved.
Image 8785 saved.
Image 8786 saved.
Image 8787 saved.
Image 8788 saved.
Image 8789 saved.
Image 8790 saved.
Image 8791 saved.
Image 8792 saved.
Image 8793 saved.
Image 8794 saved.
Image 8795 saved.
Image 8796 saved.
Image 8797 saved.
Image 8798 saved.
Image 8799 saved.
Image 8800 saved.
Image 8801 saved.
Image 8802

Image 9207 saved.
Image 9208 saved.
Image 9209 saved.
Image 9210 saved.
Image 9211 saved.
Image 9212 saved.
Image 9213 saved.
Image 9214 saved.
Image 9215 saved.
Image 9216 saved.
Image 9217 saved.
Image 9218 saved.
Image 9219 saved.
Image 9220 saved.
Image 9221 saved.
Image 9222 saved.
Image 9223 saved.
Image 9224 saved.
Image 9225 saved.
Image 9226 saved.
Image 9227 saved.
Image 9228 saved.
Image 9229 saved.
Image 9230 saved.
Image 9231 saved.
Image 9232 saved.
Image 9233 saved.
Image 9234 saved.
Image 9235 saved.
Image 9236 saved.
Image 9237 saved.
Image 9238 saved.
Image 9239 saved.
Image 9240 saved.
Image 9241 saved.
Image 9242 saved.
Image 9243 saved.
Image 9244 saved.
Image 9245 saved.
Image 9246 saved.
Image 9247 saved.
Image 9248 saved.
Image 9249 saved.
Image 9250 saved.
Image 9251 saved.
Image 9252 saved.
Image 9253 saved.
Image 9254 saved.
Image 9255 saved.
Image 9256 saved.
Image 9257 saved.
Image 9258 saved.
Image 9259 saved.
Image 9260 saved.
Image 9261 saved.
Image 9262

Image 9665 saved.
Image 9666 saved.
Image 9667 saved.
Image 9668 saved.
Image 9669 saved.
Image 9670 saved.
Image 9671 saved.
Image 9672 saved.
Image 9673 saved.
Image 9674 saved.
Image 9675 saved.
Image 9676 saved.
Image 9677 saved.
Image 9678 saved.
Image 9679 saved.
Image 9680 saved.
Image 9681 saved.
Image 9682 saved.
Image 9683 saved.
Image 9684 saved.
Image 9685 saved.
Image 9686 saved.
Image 9687 saved.
Image 9688 saved.
Image 9689 saved.
Image 9690 saved.
Image 9691 saved.
Image 9692 saved.
Image 9693 saved.
Image 9694 saved.
Image 9695 saved.
Image 9696 saved.
Image 9697 saved.
Image 9698 saved.
Image 9699 saved.
Image 9700 saved.
Image 9701 saved.
Image 9702 saved.
Image 9703 saved.
Image 9704 saved.
Image 9705 saved.
Image 9706 saved.
Image 9707 saved.
Image 9708 saved.
Image 9709 saved.
Image 9710 saved.
Image 9711 saved.
Image 9712 saved.
Image 9713 saved.
Image 9714 saved.
Image 9715 saved.
Image 9716 saved.
Image 9717 saved.
Image 9718 saved.
Image 9719 saved.
Image 9720

In [9]:
# serialize model to JSON
from keras.models import model_from_json
model_json = dcgan.combined.to_json()
with open(r"C:\Users\rishi\Documents\DCGAN Images\dcgan.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
dcgan.combined.save_weights(r"C:\Users\rishi\Documents\DCGAN Images\dcgan.h5")
print("Saved model to disk")
 


Saved model to disk
